In [1]:
import os
from google.genai import types

In [2]:
import vertexai
PROJECT_ID = os.getenv("PROJECT_ID")
REGION = os.getenv("REGION")

vertexai.init(location= REGION , project=PROJECT_ID)


In [3]:
# Create a GenerateContentConfig instance
generate_content_config = types.GenerateContentConfig(
    safety_settings=[
        types.SafetySetting(
            category="HARM_CATEGORY_HARASSMENT",
            threshold="BLOCK_ONLY_HIGH",  # Block few
        ),
        types.SafetySetting(
            category="HARM_CATEGORY_HATE_SPEECH",
            threshold="BLOCK_NONE",  # Block none
        ),
        types.SafetySetting(
            category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
            threshold="BLOCK_LOW_AND_ABOVE",  # Block most
        ),
    ],
)


In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "allymarclinicalnotesapp-0c6d62f3197f.json"

In [5]:
from google import genai

client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION)

In [6]:
from pdf2image import convert_from_path
import gc
import tempfile
import os
from google.genai import types

def ocr_and_write_page(pdf_path, page_number, dpi=300, output_path="page_text.txt"):
    images = convert_from_path(pdf_path, dpi=dpi, first_page=page_number, last_page=page_number)
    image = images[0]
    print("================================page_number",page_number)

    with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as temp_image_file:
        image.save(temp_image_file.name, format="PNG")
        temp_image_file_path = temp_image_file.name

    try:

        with open(temp_image_file_path, 'rb') as f:
            image_bytes = f.read()

        response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=[
            types.Part.from_bytes(
            data=image_bytes,
            mime_type='image/jpeg',
            ),
            'Extract the text in the image verbatim'
        ],
        )
        response=f"\n PAGE NUMBER:- {page_number}----------------------------------------\nDATA: {response.text}"
        return response
    finally:
        # Clean up
        os.remove(temp_image_file_path)
        del image
        gc.collect()

In [7]:
from pdf2image import pdfinfo_from_path

pdf_path = "Amelia_Harris_Redacted.pdf"
info = pdfinfo_from_path(pdf_path)
total_pages = info["Pages"]

In [8]:
# total_pages = 10 
for i in range(1, total_pages + 1):
    ocr_and_write_page(pdf_path=pdf_path, page_number=i, output_path=f"page_{i}_output.txt")


================================page_number 1
================================page_number 2
================================page_number 3
================================page_number 4
================================page_number 5
================================page_number 6
================================page_number 7
================================page_number 8
================================page_number 9
================================page_number 10
================================page_number 11
================================page_number 12
================================page_number 13
================================page_number 14
================================page_number 15
================================page_number 16
================================page_number 17
================================page_number 18
================================page_number 19
================================page_number 20
================================page_number 21
======================

In [9]:
responses

NameError: name 'responses' is not defined

In [ ]:
len(responses)

64